In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohamedmaher5/vehicle-classification")

print("Path to dataset files:", path)

100%|██████████| 827M/827M [00:08<00:00, 102MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mohamedmaher5/vehicle-classification/versions/1


In [3]:
! mv /root/.cache/kagglehub/datasets/mohamedmaher5/vehicle-classification/versions/1 dataset

In [4]:
DATASET_DIR = '/content/dataset/Vehicles'

In [5]:
IMAGE_SIZE = (150, 150)  # All images will be resized to this
BATCH_SIZE = 32
EPOCHS = 15 # Number of training epochs (adjust as needed)
VALIDATION_SPLIT = 0.2 # Percentage of data to use for validation

In [9]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VALIDATION_SPLIT,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [10]:
# Load training data from directory
train_generator = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 4470 images belonging to 7 classes.


In [11]:
# Load validation data from directory
validation_generator = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 1117 images belonging to 7 classes.


In [12]:
# Get the number of classes dynamically
num_classes = train_generator.num_classes

# Get the class names
class_names = list(train_generator.class_indices.keys())

print(f"\nFound {train_generator.samples} training images belonging to {num_classes} classes.")
print(f"Found {validation_generator.samples} validation images belonging to {num_classes} classes.")
print(f"Classes: {class_names}")


Found 4470 training images belonging to 7 classes.
Found 1117 validation images belonging to 7 classes.
Classes: ['Auto Rickshaws', 'Bikes', 'Cars', 'Motorcycles', 'Planes', 'Ships', 'Trains']


In [14]:
# --- Build the CNN Model ---

model = Sequential([
    # Convolutional Layer 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.2),


    # Convolutional Layer 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.3),

    #Convolutional Layer 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.3),


    # Flatten the 3D output to 1D
    Flatten(),

    # Dense Layer 1 (Hidden Layer)
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    # Output Layer
    Dense(num_classes, activation='softmax') # 'softmax' for multi-class probabilities
])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 74, 74, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 36, 36, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 17, 17, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     9,470,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,567,175 (36.50 MB)

 Trainable params: 9,566,215 (36.49 MB)

 Non-trainable params: 960 (3.75 KB)

In [15]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
  1/140 ━━━━━━━━━━━━━━━━━━━━ 27:57 12s/step - accuracy: 0.1562 - loss: 3.2540

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


140/140 ━━━━━━━━━━━━━━━━━━━━ 88s 548ms/step - accuracy: 0.3673 - loss: 2.2263 - val_accuracy: 0.1423 - val_loss: 3.2062
Epoch 2/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 127s 493ms/step - accuracy: 0.5325 - loss: 1.4008 - val_accuracy: 0.2149 - val_loss: 3.3207
Epoch 3/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 82s 499ms/step - accuracy: 0.6266 - loss: 1.0923 - val_accuracy: 0.2113 - val_loss: 2.3752
Epoch 4/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 68s 489ms/step - accuracy: 0.6492 - loss: 0.9963 - val_accuracy: 0.2346 - val_loss: 2.5458
Epoch 5/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 69s 491ms/step - accuracy: 0.6817 - loss: 0.9081 - val_accuracy: 0.6965 - val_loss: 0.8278
Epoch 6/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 69s 490ms/step - accuracy: 0.7193 - loss: 0.7968 - val_accuracy: 0.7126 - val_loss: 0.8023
Epoch 7/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 69s 490ms/step - accuracy: 0.7331 - loss: 0.7707 - val_accuracy: 0.6553 - val_loss: 0.8871
Epoch 8/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 68s 488ms/step - accuracy: 0.7386 - loss: 0.7417 - va

In [34]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VALIDATION_SPLIT,
)

In [35]:
# Load training data from directory
train_generator = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 4470 images belonging to 7 classes.


In [36]:
# Load validation data from directory
validation_generator = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 1117 images belonging to 7 classes.


In [38]:
base_model = MobileNetV2(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
                         include_top=False, # Don't include the classification layer
                         weights='imagenet')

<ipython-input-38-190d023a607f>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),


In [39]:
base_model.trainable = False

In [40]:
MobileNetV2_model = Sequential([

    base_model, #MobileNetV2

    GlobalAveragePooling2D(),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(7, activation='softmax')
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 74, 74, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 36, 36, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 17, 17, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     9,470,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,567,175 (36.50 MB)

 Trainable params: 9,566,215 (36.49 MB)

 Non-trainable params: 960 (3.75 KB)

In [41]:
MobileNetV2_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [42]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [43]:
history = MobileNetV2_model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

Epoch 1/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 58s 359ms/step - accuracy: 0.7569 - loss: 0.7644 - val_accuracy: 0.9508 - val_loss: 0.1245
Epoch 2/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 42s 303ms/step - accuracy: 0.9568 - loss: 0.1269 - val_accuracy: 0.9579 - val_loss: 0.1054
Epoch 3/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 42s 298ms/step - accuracy: 0.9669 - loss: 0.1103 - val_accuracy: 0.9588 - val_loss: 0.1088
Epoch 4/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 41s 296ms/step - accuracy: 0.9672 - loss: 0.1041 - val_accuracy: 0.9678 - val_loss: 0.1004
Epoch 5/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 42s 300ms/step - accuracy: 0.9823 - loss: 0.0581 - val_accuracy: 0.9687 - val_loss: 0.0845
Epoch 6/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 41s 292ms/step - accuracy: 0.9870 - loss: 0.0401 - val_accuracy: 0.9696 - val_loss: 0.0878
Epoch 7/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 41s 296ms/step - accuracy: 0.9876 - loss: 0.0317 - val_accuracy: 0.9714 - val_loss: 0.0874
Epoch 8/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 42s 301ms/step - accuracy: 0.9938 - loss: 0

In [47]:
loss, accuracy = MobileNetV2_model.evaluate(validation_generator)

35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 215ms/step - accuracy: 0.9663 - loss: 0.0842


In [48]:
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Loss: 0.0845
Validation Accuracy: 0.9687
